In [ ]:
from pymatgen.core import Lattice, Structure
from ase.io import read, write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter
import numpy as np
import copy
import matplotlib.pyplot as plt
import MDAnalysis as mda
from MDAnalysis.analysis.rdf import InterRDF
from scipy.stats import entropy
import math
import statistics

In [ ]:
file_loc = r"path_to_supercell.cif" #change file path
structure = read(file_loc)

In [ ]:
structure

In [ ]:
def struct_gen(n, m, structure, struc_list):
    
    atom_type = structure.get_atomic_numbers() #list of atomic numbers at each index
    
    Ga_index = [] #list of indices of Ga
    O_index = [] #list of indices of O
    for i in range(len(atom_type)):
        if atom_type[i] == 31:
            Ga_index.append(i)
        if atom_type[i] == 8:
            O_index.append(i)
    
    Ga_subs_list = [] #list of Ga indices to be substituted
    O_vac_list = [] #list of indices for O vacancy
    for i in range(m):
        Ga_subs_list.append(np.random.permutation(len(Ga_index))[:int(2*n)].tolist())
        O_vac_list.append(np.random.permutation(len(O_index))[:n].tolist())
    
    Ga_subs_index = [] #list of indices for substituted atoms
    O_vac_index = [] #list of indices for vacancies
    temp = copy.deepcopy(structure)

    #generating structures by substitution
    for i in range(m): 
        atom_type = structure.get_atomic_numbers() 
        subs_index = []
        vac_index = []
        for j in Ga_subs_list[i]:
            atom_type[Ga_index[j]] = 12
            subs_index.append(Ga_index[j])
        for k in O_vac_list[i]:
            atom_type[O_index[k]] = 0 #placeholder 'X' will have to delete these sites manually later to create vacancies
            vac_index.append(O_index[k])
        temp.numbers = atom_type 
        struc_list.append(temp.copy()) 
        Ga_subs_index.append(subs_index)
        O_vac_index.append(vac_index)
    
    return struc_list, Ga_subs_index, O_vac_index

In [ ]:
def struct_equ(m, struc_list, Ga_subs_index, O_vac_index):
    
    Mg_distances = [] #list of all pairwise distances between substituted atoms for each structure
    vac_distances = [] #list of all pairwise distances between vacancies for each structure
    Mg_vac_distances = [] #list all of pairwise distances between substitution site and vacancy site for each structure

    #calculating distances
    for i in range(m):
        subs_dist = []
        vacs_dist = []
        mix_dist = []
        for j in range(len(Ga_subs_index[i])):
            for k in range(j+1,len(Ga_subs_index[i])):
                pos1 = struc_list[i].positions[Ga_subs_index[i][j]]
                pos2 = struc_list[i].positions[Ga_subs_index[i][k]]
                subs_dist.append(((pos2[0]-pos1[0])**2 + (pos2[1]-pos1[1])**2 + (pos2[2]-pos1[2])**2)**0.5)
            for l in range(len(O_vac_index[i])):
                pos1 = struc_list[i].positions[Ga_subs_index[i][j]]
                pos2 = struc_list[i].positions[O_vac_index[i][l]]
                mix_dist.append(((pos2[0]-pos1[0])**2 + (pos2[1]-pos1[1])**2 + (pos2[2]-pos1[2])**2)**0.5)
        for j in range(len(O_vac_index[i])):
            for k in range(j+1,len(O_vac_index[i])):
                pos1 = struc_list[i].positions[O_vac_index[i][j]]
                pos2 = struc_list[i].positions[O_vac_index[i][k]]
                vacs_dist.append(((pos2[0]-pos1[0])**2 + (pos2[1]-pos1[1])**2 + (pos2[2]-pos1[2])**2)**0.5)
        Mg_distances.append(subs_dist)
        vac_distances.append(vacs_dist)
        Mg_vac_distances.append(mix_dist)

    equivalent = [] #list of structures that are equivalent 
    #structures are considered equivalent if all pairwise MgMg distances and MgX distances are the same for 2 strctures due to periodic cell
    for i in range(m):
        if i not in equivalent:
            for j in range(i+1,m):
                for k in abs((np.sort(Mg_distances[i])-np.sort(Mg_distances[j])))<0.000001:
                    if k == False:
                        break
                if k == True:
                    print(i,"equivalent to",j)
                    equivalent.append(j)
            for j in range(i+1,m):
                for k in abs((np.sort(Mg_vac_distances[i])-np.sort(Mg_vac_distances[j])))<0.000001:
                    if k == False:
                        break
                if k == True:
                    print(i,"equivalent to",j)
                    equivalent.append(j)

    equivalent.sort(reverse = True)
    print("equivalent =", equivalent)
    #delete all equivalent structures
    for i in equivalent:
        del struc_list[i]
        del Ga_subs_index[i]
        del O_vac_index[i]
    n_equ = len(equivalent)
    
    return struc_list, n_equ, Ga_subs_index, O_vac_index, Mg_distances

In [ ]:
nsub = 8
ts = 1000
subs_structs = []

subs_structs, subs_sites, vac_sites = struct_gen(nsub, ts, structure, subs_structs)
subs_structs, equ, subs_sites, vac_sites, mgdist = struct_equ(ts, subs_structs, subs_sites, vac_sites)
print("equ =",equ)

#replace deleted equivalent structures to retain total no. of structures = ts
while equ != 0:
    subs_structs, new_subs_sites, new_vac_sites = struct_gen(nsub, equ, structure, subs_structs)
    subs_sites.extend(new_subs_sites)
    vac_sites.extend(new_vac_sites)
    subs_structs, equ, subs_sites, vac_sites, mgdist = struct_equ(ts, subs_structs, subs_sites, vac_sites)
    print("no. equ =",equ)

In [ ]:
subs_structs #list of substituted structures

In [ ]:
#Site specific substitution was achieved by replacing all Ga atoms occupying a specific site type with a placeholder atom
#Here all such placeholder atoms are replaced with Ga
for i in range(ts):
    atoms = subs_structs[i].get_atomic_numbers()
    for j in range(len(atoms)):
        if atoms[j] == 52:
            atoms[j] = 31
    subs_structs[i].numbers = atoms

In [ ]:
u = []

for i in range(len(subs_structs)):
    n_atoms = len(subs_structs[i])
    u.append(mda.Universe.empty(n_atoms, trajectory=True))  # Create empty Universe
    u[i].add_TopologyAttr("names", subs_structs[i].get_chemical_symbols())  # Add atom types/names
    u[i].atoms.positions = subs_structs[i].get_positions()  # Add positions
    u[i].dimensions = subs_structs[i].cell.cellpar()  # Add box dimensions

specie = ["name Ga", "name Mg", "name O", "name X"]
rdf = []
bins = []

#calculate pairwise radial distribution functions

for i in u:
    pairs = {}
    for j in range(len(specie)):
        group_a = i.select_atoms(specie[j])  # Select atoms labeled "A"
        for k in range(j, len(specie)):
            group_b = i.select_atoms(specie[k])  # Select atoms labeled "B"
            pdf = InterRDF(group_a, group_b, range=(1.5, 5.0), nbins=100) #PDF of A-B
            pdf.run()
            if k == j:
                pdf.results.rdf[0] = 0.0
            pairs[specie[j][5:]+"-"+specie[k][5:]] = pdf.results.rdf
    rdf.append(pairs)
bins = pdf.results.bins

In [ ]:
def normalize(data): 
    return (data - data.min()) / (data.max() - data.min())

In [ ]:
avgRDF = {'Ga-Ga': np.zeros(100), 'Ga-Mg': np.zeros(100), 'Ga-O': np.zeros(100), 'Ga-X': np.zeros(100), 'Mg-Mg': np.zeros(100), 'Mg-O': np.zeros(100), 'Mg-X': np.zeros(100), 'O-O': np.zeros(100), 'O-X': np.zeros(100), 'X-X': np.zeros(100)}

ele = ['Ga','Mg','O','X']

for i in rdf:
    for j in range(len(ele)):
        for k in range(j, len(ele)):
            avgRDF[ele[j]+"-"+ele[k]] = np.add(avgRDF[ele[j]+"-"+ele[k]], i[ele[j]+"-"+ele[k]])

for i in avgRDF:
    avgRDF[i] = normalize(avgRDF[i]/len(rdf)) #normalize average RDF

In [ ]:
#select 100 random samples of 10 structures
sample = []
for i in range(100):
    sample.append(np.random.permutation(1000)[:10].tolist())

In [ ]:
sampRDF = []

for i in sample:
    PDF = {'Ga-Ga': np.zeros(100), 'Ga-Mg': np.zeros(100), 'Ga-O': np.zeros(100), 'Ga-X': np.zeros(100), 'Mg-Mg': np.zeros(100), 'Mg-O': np.zeros(100), 'Mg-X': np.zeros(100), 'O-O': np.zeros(100), 'O-X': np.zeros(100), 'X-X': np.zeros(100)}
    for j in i:
        for k in range(len(ele)):
            for l in range(k, len(ele)):
                PDF[ele[k]+"-"+ele[l]] = np.add(PDF[ele[k]+"-"+ele[l]], rdf[j][ele[k]+"-"+ele[l]])
    sampRDF.append(PDF)
    
for sampno in sampRDF:
    for j in sampno:
        sampno[j] = normalize(sampno[j]/len(sample[0])) #normalized sample average of RDFs

In [ ]:
#plot to compare sample average of RDFs to global average of RDFs
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10)) = plt.subplots(nrows= 5, ncols=2, figsize=(15,30))

fig.suptitle("Partial RDFs for β-Ga₂O₃ with 25% of tetrahedral Ga sites substituted with Mg", fontsize = 20)


for i in range(len(sampRDF)):
    ax1.plot(bins,sampRDF[i]['Ga-Ga'], linewidth = 1)
ax1.plot(bins, avgRDF['Ga-Ga'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax1.set_xlabel("Distance (Å)")
ax1.set_ylabel("g(r)")
ax1.set_title("Ga-Ga Partial RDF")
ax1.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax2.plot(bins,sampRDF[i]['Ga-Mg'], linewidth = 1)
ax2.plot(bins, avgRDF['Ga-Mg'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax2.set_xlabel("Distance (Å)")
ax2.set_ylabel("g(r)")
ax2.set_title("Ga-Mg Partial RDF")
ax2.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax3.plot(bins,sampRDF[i]['Ga-O'], linewidth = 1)
ax3.plot(bins, avgRDF['Ga-O'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax3.set_xlabel("Distance (Å)")
ax3.set_ylabel("g(r)")
ax3.set_title("Ga-O Partial RDF")
ax3.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax4.plot(bins,sampRDF[i]['Ga-X'], linewidth = 1)
ax4.plot(bins, avgRDF['Ga-X'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax4.set_xlabel("Distance (Å)")
ax4.set_ylabel("g(r)")
ax4.set_title("Ga-vacancy Partial RDF")
ax4.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax5.plot(bins,sampRDF[i]['Mg-Mg'], linewidth = 1)
ax5.plot(bins, avgRDF['Mg-Mg'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax5.set_xlabel("Distance (Å)")
ax5.set_ylabel("g(r)")
ax5.set_title("Mg-Mg Partial RDF")
ax5.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax6.plot(bins,sampRDF[i]['Mg-O'], linewidth = 1)
ax6.plot(bins, avgRDF['Mg-O'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax6.set_xlabel("Distance (Å)")
ax6.set_ylabel("g(r)")
ax6.set_title("Mg-O Partial RDF")
ax6.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax7.plot(bins,sampRDF[i]['Mg-X'], linewidth = 1)
ax7.plot(bins, avgRDF['Mg-X'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax7.set_xlabel("Distance (Å)")
ax7.set_ylabel("g(r)")
ax7.set_title("Mg-vacancy Partial RDF")
ax7.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax8.plot(bins,sampRDF[i]['O-O'], linewidth = 1)
ax8.plot(bins, avgRDF['O-O'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax8.set_xlabel("Distance (Å)")
ax8.set_ylabel("g(r)")
ax8.set_title("O-O Partial RDF")
ax8.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax9.plot(bins,sampRDF[i]['O-X'], linewidth = 1)
ax9.plot(bins, avgRDF['O-X'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax9.set_xlabel("Distance (Å)")
ax9.set_ylabel("g(r)")
ax9.set_title("O-vacancy Partial RDF")
ax9.legend(fontsize=12)

for i in range(len(sampRDF)):
    ax10.plot(bins,sampRDF[i]['X-X'], linewidth = 1)
ax10.plot(bins, avgRDF['X-X'], linestyle='-.', color = 'black', linewidth = 2, label = "full-set average")
ax10.set_xlabel("Distance (Å)")
ax10.set_ylabel("g(r)")
ax10.set_title("vacancy-vacancy Partial RDF")
ax10.legend(fontsize=12)


plt.subplots_adjust(top=0.95)
#plt.savefig("RDFsgamma25.png")

In [ ]:
kldiv = []

for i in sampRDF:
    div = {}
    for j in range(len(ele)):
        for k in range(j, len(ele)):
            div[ele[j]+"-"+ele[k]] = entropy(i[ele[j]+"-"+ele[k]], avgRDF[ele[j]+"-"+ele[k]])
    kldiv.append(div) #compute KL divergence of sample averages from global average

In [ ]:
#split KL divergence by pairs
klGaGa = []
klGaMg = []
klGaO = []
klGaX = []
klMgMg = []
klMgO = []
klMgX = []
klOO = []
klOX = []
klXX = []

for i in kldiv:
    klGaGa.append(i['Ga-Ga'])
    klGaMg.append(i['Ga-Mg'])
    klGaO.append(i['Ga-O'])
    klGaX.append(i['Ga-X'])
    klMgMg.append(i['Mg-Mg'])
    klMgO.append(i['Mg-O'])
    klMgX.append(i['Mg-X'])
    klOO.append(i['O-O'])
    klOX.append(i['O-X'])
    klXX.append(i['X-X'])

In [ ]:
print("GaGa: min", min(klGaGa), "max", max(klGaGa), "range", (max(klGaGa) - min(klGaGa)))
print("GaMg: min", min(klGaMg), "max", max(klGaMg), "range", (max(klGaMg) - min(klGaMg)))
print("GaO: min", min(klGaO), "max", max(klGaO), "range", (max(klGaO) - min(klGaO)))
print("GaX: min", min(klGaX), "max", max(klGaX), "range", (max(klGaX) - min(klGaX)))
print("MgMg: min", min(klMgMg), "max", max(klMgMg), "range", (max(klMgMg) - min(klMgMg)))
print("MgO: min", min(klMgO), "max", max(klMgO), "range", (max(klMgO) - min(klMgO)))
print("MgX: min", min(klMgX), "max", max(klMgX), "range", (max(klMgX) - min(klMgX)))
print("OO: min", min(klOO), "max", max(klOO), "range", (max(klOO) - min(klOO)))
print("OX: min", min(klOX), "max", max(klOX), "range", (max(klOX) - min(klOX)))
print("XX: min", min(klXX), "max", max(klXX), "range", (max(klXX) - min(klXX)))

In [ ]:
wtdKL = []
GaGa = math.comb(48,2) # No. of Ga-Ga pairs
GaMg = 48*16
GaO = 48*88
GaX = 48*8 
MgMg = math.comb(16,2)
MgO = 16*88
MgX = 16*8
OO = math.comb(88,2)
OX = 88*8
XX = math.comb(8,2)

for i in range(len(kldiv)):
    #removed XX kl-div below for 6.25% kappa
    num = (GaGa*klGaGa[i] + GaMg*klGaMg[i] + GaO*klGaO[i] + GaX*klGaX[i] + MgMg*klMgMg[i] + MgO*klMgO[i] + MgX*klMgX[i] + OO*klOO[i] + OX*klOX[i] + XX*klXX[i])
    den = (GaGa + GaMg + GaO + GaX + MgMg + MgO + MgX + OO + OX + XX)
    wtdKL.append(num/den) #calculate weighted KL divergence (weights correspond to the number of occurrences of each pair type in the structure)

In [ ]:
sample[wtdKL.index(min(wtdKL))] #sample with lowest weighted KL divergence

In [ ]:
print(min(wtdKL))
print(statistics.mean(wtdKL))
print(statistics.median(wtdKL))

In [ ]:
#histogram of pairwise KL divergences for all substituted structures
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10)) = plt.subplots(nrows= 5, ncols=2, figsize=(15,30))

fig.suptitle("Sample size = 10", fontsize = 30)

ax1.hist(klGaGa, bins = np.arange(0,1, 0.0005), width = 0.005)
ax1.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax1.set_title("KL divergence values for Ga-Ga")

ax2.hist(klGaMg, bins = np.arange(0,1, 0.0005), width = 0.005)
ax2.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax2.set_title("KL divergence values for Ga-Mg")

ax3.hist(klMgMg, bins = np.arange(0,1, 0.0005), width = 0.005)
ax3.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax3.set_title("KL divergence values for Mg-Mg")

ax4.hist(klGaO, bins = np.arange(0,1, 0.0005), width = 0.005)
ax4.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax4.set_title("KL divergence values for Ga-O")

ax5.hist(klGaX, bins = np.arange(0,1, 0.0005), width = 0.005)
ax5.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax5.set_title("KL divergence values for Ga-Vacancy")

ax6.hist(klMgO, bins = np.arange(0,1, 0.0005), width = 0.005)
ax6.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax6.set_title("KL divergence values for Mg-O")

ax7.hist(klMgX, bins = np.arange(0,1, 0.0005), width = 0.005)
ax7.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax7.set_title("KL divergence values for Mg-X")

ax8.hist(klOO, bins = np.arange(0,1, 0.0005), width = 0.005)
ax8.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax8.set_title("KL divergence values for O-O")

ax9.hist(klOX, bins = np.arange(0,1, 0.0005), width = 0.005)
ax9.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax9.set_title("KL divergence values for O-Vacancy")

ax10.hist(klXX, bins = np.arange(0,1, 0.0005), width = 0.005)
ax10.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
ax10.set_title("KL divergence values for Vacancy-Vacancy")

plt.subplots_adjust(top=0.95)
#plt.savefig("KLdivergence Histogram sample size 20.jpg")

In [ ]:
#remove placeholder 'X' to create vacancies
for i in range(ts):
    del subs_structs[i][vac_sites[i]]

In [ ]:
#save structures from sample with lowest weighted KL diveregence
for i in sample[wtdKL.index(min(wtdKL))]:
    output_loc = r"path/to/output/stuructures"+str(i)+".cif" #change file path
    write(output_loc, subs_structs[i])